In [ ]:
import torch
import torch.nn as nn
import torch.optim
import torchvision
import torch.utils.data
import numpy as np 
import matplotlib.pyplot as plt
import Autoencoder as AE
import scipy
import scipy.interpolate
import os 

# use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Reproducibility  
torch.manual_seed(0);

In [ ]:
### Load and Process Data ###
run_interp = False 
cyl_data = np.load('./SamsStuff/data_numpy.npy') # of shape [n_samples, (u, v, vort), n_gridpoints]
xy_coords = np.load('./SamsStuff/xy_coords.npy').T # of shape [n_gridpoints, (x, y)].T
vort_data = cyl_data[:, 2, :]

# interpolate onto structured grid
x = xy_coords[0, :]
y = xy_coords[1, :]
x_bounds = [x.min(), x.max()]
y_bounds = [y.min(), y.max()]
scaling = 20 #controls resolution of structured grid
Nx = int((x_bounds[1] - x_bounds[0]) * scaling)
Ny = int((y_bounds[1] - y_bounds[0]) * scaling)
grid_x, grid_y = np.meshgrid(np.linspace(x_bounds[0], x_bounds[1], Nx), np.linspace(y_bounds[0], y_bounds[1], Ny))

# cylinder mask 
mask = np.ones_like(grid_x)
mask[np.where(grid_x**2 + grid_y**2 < 0.5**2)] = 0

if run_interp: # takes ~3m for scaling = 20
    vort_structured = np.zeros((vort_data.shape[0], Ny, Nx))
    for i in range(vort_data.shape[0]):
        interpolated = scipy.interpolate.griddata((x,y), vort_data[i], (grid_x, grid_y), method='cubic')
        vort_structured[i] = interpolated * mask
    np.save('vort_structured{}.npy'.format(scaling), vort_structured)
else:
    vort_structured = np.load('vort_structured{}.npy'.format(scaling))

im_size = vort_structured.shape[1] * vort_structured.shape[2]

### Dataloaders ###
test_perc = 0.1
split_idx = int((1-test_perc) * vort_structured.shape[1])

batch_size = 128
vort_torch = torch.from_numpy(vort_structured).float()
vort_dset = torch.utils.data.TensorDataset(vort_torch[split_idx:, :, :])
dataloader = torch.utils.data.DataLoader(vort_dset, batch_size=batch_size, shuffle=False)

vort_dset_test = torch.utils.data.TensorDataset(vort_torch[:split_idx, :, :])
dataloader_test = torch.utils.data.DataLoader(vort_dset, batch_size=batch_size, shuffle=False)


In [ ]:
from optimizers import GD

In [ ]:
optimizers = [
    'Adam',
    #'Adam_beta0.5',
    #GD
]

# Training parameters
lr_schedulers = [
    #'CosineAnnealingWarmRestarts', 
    #'ReduceLROnPlateau', 
    'Armijo'
]
EPOCHS = 2
initial_lr = 0.001
armijo = False
SAVE_DIR = './train_results_cyl/trial4/'

val_losses = {}
for opt in optimizers:
    for sched in lr_schedulers:
        #create directory for results
        out_dir = SAVE_DIR + opt + '/'
        os.makedirs(out_dir, exist_ok=True)

        print('\nTraining with optimizer: {}    lr = {}'.format(opt, sched))
        #Init network
        n_latent = int(0.01* cyl_data.shape[-1]) #use 1% of original amount of data
        n_hidden = 5 #number of hidden layers 
        NN_width = 1024*4  #width of the hidden layers
        error = nn.L1Loss()
        net = AE.AutoEncoder(im_size, n_latent, n_hidden, NN_width, taper = True, square=False).to(device)

        if opt == 'GD':
            optimizer = torch.optim.SGD(net.parameters(), lr=initial_lr)
        elif opt == 'Adam':
            optimizer = torch.optim.Adam(net.parameters(), betas=(0.9, 0.999), lr=initial_lr)
        elif opt == 'Adam_beta0.5':
            optimizer = torch.optim.Adam(net.parameters(), betas=(0.5, 0.999), lr=initial_lr)

        if sched == 'CosineAnnealingWarmRestarts':
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 200)
            use_loss = False
        elif sched == 'ReduceLROnPlateau':
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience  = 7)
            use_loss = True
        elif sched == 'Armijo':
            scheduler = None
            use_loss = False
            armijo = True


        #train
        train_out = AE.train_cyl(net, optimizer, dataloader, dataloader_test, epochs=EPOCHS, scheduler = scheduler, armijo = armijo, use_loss = use_loss)

        #save results
        out_fname = out_dir + 'train_results_{}_lr={}.png'.format(train_out['optimizer'], sched)
        AE.plot_cyl_reconst(out_fname, train_out['net'], dataloader_test, Nx, Ny, grid_x, grid_y)

        #plot loss curve 
        plt.figure(figsize = (8,6))
        plt.plot(train_out['losses'])
        plt.xlabel('Iterations')
        plt.ylabel('L1 Loss')
        plt.yscale('log')
        plt.savefig(out_dir + 'loss_curve_{}_lr={}.png'.format(train_out['optimizer'], sched))
        plt.close()

        #save losses
        np.save(out_dir + 'losses_{}_lr={}.npy'.format(train_out['optimizer'], sched), train_out['losses'])

        val_losses[opt + '_' + str(sched)] = train_out['test_error']

        #empty cuda cache
        torch.cuda.empty_cache()

In [ ]:
# print sorted val losses dictionary
sorted_val_losses = {k: v for k, v in sorted(val_losses.items(), key=lambda item: item[1])}
np.save(SAVE_DIR+'val_losses.npy', sorted_val_losses) #np.load(SAVE_DIR+'val_losses.npy', allow_pickle=True).item()
sorted_val_losses
